In [1]:
import deepchem
deepchem.__version__

'2.5.0'

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
os.environ['CUDA_LAUNCH_BLOCKING']="1" 

from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==3])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")


 /device:CPU:0 || Unnamed device || CPU || 256.0 MiB
 /device:GPU:0 || Unnamed device || GPU || 44.8 GiB


In [3]:
#MTL#
import tempfile
import deepchem as dc
import pandas as pd
from sklearn.metrics import precision_score
import tensorflow as tf
from tensorflow import keras
import time
import psutil
import warnings
warnings.filterwarnings("ignore")
import torch
import random
import tensorflow as tf
import numpy as np

set_seed(seed=8, tensorflow=True)

epoch_number=['10','100','200','500']
#,'100','200','500']

for e_n in epoch_number:
    print(e_n)
    

    start = time.time()

    ###result####
    number = list(range(0,500))    
    result_df=pd.DataFrame(index=number )
    col_list=['Task',
            'Train AUC', 'Train ACC','Train Balance ACC','Train F1','Train Precision', 'Train Recall','Train Specificity',
            'Valid AUC', 'Valid ACC','Valid Balance ACC','Valid F1','Valid Precision', 'Valid Recall','Valid Specificity',
            'Test AUC', 'Test ACC', 'Test Balance ACC','Test F1', 'Test Precision','Test Recall','Test Specificity']

    df=pd.read_csv('Zebrafish_final202310_withMorphologyALLdata.csv')
    tasks=['LEL_MORT','LEL_YSE','LEL_AXIS','LEL_EYE','LEL_SNOU', 'LEL_JAW',
        'LEL_OTIC','LEL_PE','LEL_BRAI','LEL_SOMI','LEL_PFIN','LEL_CFIN',
        'LEL_PIG','LEL_CIRC', 'LEL_TRUN','LEL_SWIM','LEL_NC','LEL_TR', '18_END_LEC', 'MOR_LEC','SUBLETH_17_END_LEC', 'TOX_SCO',
        'MO24', 'DP24', 'SM24', 'NC24','MORT','YSE', 'AXIS', 'EYE',
        'SNOU', 'JAW', 'OTIC', 'PE', 'BRAI', 'SOMI', 'PFIN', 'CFIN',
        'PIG','CIRC', 'TRUN', 'SWIM','NC','TR','MOV21', 'AUC21','METAB','MIC']
    #task=task.drop(columns='Unnamed: 0')
    labels=tasks
    for i in df.index:
        for q in labels:
            try:
                if df.loc[i,q]== 'ACT':
                    df.loc[i,q]=int(1)
                if df.loc[i,q]== 'INACT':           
                    df.loc[i,q]=int(0)             
            except:
                pass   

    print('ECFP6')
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
      df.to_csv(tmpfile.name)
      loader = dc.data.CSVLoader(tasks, feature_field="SMILES",
                    featurizer=dc.feat.CircularFingerprint(size=1024, radius=3))
      dataset = loader.create_dataset(tmpfile.name)
    len(dataset)
    splitter = dc.splits.RandomStratifiedSplitter()
    # Splitting dataset into train and test datasets
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset,frac_train=0.7, frac_valid=0.1, frac_test=0.2,seed=8)
    n_tasks = len(tasks)
    n_features = train_dataset.get_data_shape()[0]
    
    print('train')
    model = dc.models.MultitaskClassifier(n_tasks, 
        n_features,
       momentum= 0.9,
       learning_rate=0.0001,
       dropouts=[0.25,0.15,0.1],
       layer_sizes=[200,100,50],
       penalty=0.005,
       weight_decay_penalty_type='l2')
    
    e_n=int(e_n)
    # Open a strategy scope.
    for d in ['/device:GPU:2','/device:GPU:3','/device:GPU:4','/device:GPU:5']:
        model.fit(train_dataset,nb_epoch=e_n)
    #######

    print('eval')
    y_true_train=train_dataset.y
    y_tr = model.predict(train_dataset)
    y_tr_label = (np.array(y_tr) >= 0.5).astype(int)

    
    y_true_valid = valid_dataset.y
    y_valid = model.predict(valid_dataset)
    y_valid_label = (np.array(y_valid) >= 0.5).astype(int)

    y_true = test_dataset.y
    y_pred = model.predict(test_dataset)
    y_pred_label = (np.array(y_pred) >= 0.5).astype(int)

    AUC_metric = dc.metrics.roc_auc_score
    ACC_metric = dc.metrics.accuracy_score
    PR_metric = dc.metrics.precision_score
    RC_metric = dc.metrics.recall_score
    F1_metric = dc.metrics.f1_score
    SP_metric = dc.metrics.recall_score


    for i in range(n_tasks):
        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true_train[:,i], y_tr_label[:,i][:,1],pos_label=1)
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i], average=None)[1]
        SP_score = SP_metric(y_true_train[:,i], y_tr_label[:,i][:,1],pos_label=0)
        
        result_df.loc[i,'Task']=tasks[i]
        result_df.loc[i,'Train AUC']=AUC_score
        result_df.loc[i,'Train ACC']=ACC_score
        result_df.loc[i,'Train F1']=F1_score
        result_df.loc[i,'Train Precision']=PR_score
        result_df.loc[i,'Train Recall']=RC_score    
        result_df.loc[i,'Train Specificity']=SP_score    
        
        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true_valid[:,i]),y_valid_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true_valid[:,i], y_valid_label[:,i][:,1],pos_label=1)
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid_label[:,i], average=None)[1]
        SP_score = SP_metric(y_true_valid[:,i], y_valid_label[:,i][:,1],pos_label=0)

        result_df.loc[i,'Valid AUC']=AUC_score
        result_df.loc[i,'Valid ACC']=ACC_score
        result_df.loc[i,'Valid F1']=F1_score
        result_df.loc[i,'Valid Precision']=PR_score
        result_df.loc[i,'Valid Recall']=RC_score    
        result_df.loc[i,'Valid Specificity']=SP_score    

        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i], average=None)[1]
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true[:,i], y_pred_label[:,i][:,1],pos_label=1)
        SP_score = SP_metric(y_true[:,i], y_pred_label[:,i][:,1],pos_label=0)

        result_df.loc[i,'Test AUC']=AUC_score
        result_df.loc[i,'Test ACC']=ACC_score
        result_df.loc[i,'Test F1']=F1_score
        result_df.loc[i,'Test Precision']=PR_score
        result_df.loc[i,'Test Recall']=RC_score
        result_df.loc[i,'Test Specificity']=SP_score    

    mean=result_df.mean()
    mean=mean.tolist()
    mean.insert(0,'AVG')
    column_headers = list(result_df.columns.values)
    for i in range(len(column_headers)):
        result_df.iloc[48,i]=mean[i]    
    result_df=result_df.dropna()
    end = time.time()
    print('Time ',end - start)
    
    result_df.to_csv('DChem_FinalFishData_MTL_LS215_'+str(e_n)+'.csv')  

10
ECFP6
train
eval
執行時間:  84.54695558547974
100
ECFP6
train
eval
執行時間:  523.7280149459839
200
ECFP6
train
eval
執行時間:  1041.9083275794983
500
ECFP6
train
eval
執行時間:  2566.5468702316284


In [ ]:
import tempfile
import deepchem as dc
import pandas as pd
from sklearn.metrics import precision_score
import tensorflow as tf
from tensorflow import keras
import time
import psutil
import warnings
warnings.filterwarnings("ignore")
import torch
import random
import tensorflow as tf
import numpy as np

set_seed(seed=8, tensorflow=True, pytorch=True)


epoch_number=['10','100','200','500']


for e_n in epoch_number:
    print(e_n)

    start = time.time()
    ###result####
    number = list(range(0,500))    
    result_df=pd.DataFrame(index=number )
    col_list=['Task',
            'Train AUC', 'Train ACC','Train Balance ACC','Train F1','Train Precision', 'Train Recall','Train Specificity',
            'Valid AUC', 'Valid ACC','Valid Balance ACC','Valid F1','Valid Precision', 'Valid Recall','Valid Specificity',
            'Test AUC', 'Test ACC', 'Test Balance ACC','Test F1', 'Test Precision','Test Recall','Test Specificity']

    df=pd.read_csv('Zebrafish_final202310_withMorphologyALLdata.csv')
    tasks=['LEL_MORT','LEL_YSE','LEL_AXIS','LEL_EYE','LEL_SNOU', 'LEL_JAW',
        'LEL_OTIC','LEL_PE','LEL_BRAI','LEL_SOMI','LEL_PFIN','LEL_CFIN',
        'LEL_PIG','LEL_CIRC', 'LEL_TRUN','LEL_SWIM','LEL_NC','LEL_TR', '18_END_LEC', 'MOR_LEC','SUBLETH_17_END_LEC', 'TOX_SCO',
        'MO24', 'DP24', 'SM24', 'NC24','MORT','YSE', 'AXIS', 'EYE',
        'SNOU', 'JAW', 'OTIC', 'PE', 'BRAI', 'SOMI', 'PFIN', 'CFIN',
        'PIG','CIRC', 'TRUN', 'SWIM','NC','TR','MOV21', 'AUC21','METAB','MIC']
    #task=task.drop(columns='Unnamed: 0')
    labels=tasks
    for i in df.index:
        for q in labels:
            try:
                if df.loc[i,q]== 'ACT':
                    df.loc[i,q]=int(1)
                if df.loc[i,q]== 'INACT':           
                    df.loc[i,q]=int(0)             
            except:
                pass   

    print('ECFP6')
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
      df.to_csv(tmpfile.name)
      loader = dc.data.CSVLoader(tasks, feature_field="SMILES",
                    featurizer=dc.feat.CircularFingerprint(size=1024, radius=3))
      dataset = loader.create_dataset(tmpfile.name)

    len(dataset)

    splitter = dc.splits.RandomStratifiedSplitter()
    # Splitting dataset into train and test datasets
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset,frac_train=0.7, frac_valid=0.1, frac_test=0.2,seed=8)

    n_tasks = len(tasks)
    n_features = train_dataset.get_data_shape()[0]
    
    print('train')
    model = dc.models.RobustMultitaskClassifier(n_tasks, 
        n_features,
       momentum= 0.9,
       learning_rate=0.0001,
       dropouts=[0.25,0.15,0.1],
       #[400,100,50]                                         
       layer_sizes=[400,200,100],
       penalty=0.005,
       weight_decay_penalty_type='l2')
    
    e_n=int(e_n)
    # Open a strategy scope.
    for d in ['/device:GPU:0']:
        model.fit(train_dataset,nb_epoch=e_n)
    #######

    print('eval')
    y_true_train=train_dataset.y
    y_tr = model.predict(train_dataset)
    y_tr_label = (y_tr >= 0.5) 

    y_true_valid = valid_dataset.y
    y_valid = model.predict(valid_dataset)
    y_valid_label = (y_valid >= 0.5) 

    y_true = test_dataset.y
    y_pred = model.predict(test_dataset)
    y_pred_label = (y_pred >= 0.5) 

    AUC_metric = dc.metrics.roc_auc_score
    ACC_metric = dc.metrics.accuracy_score
    PR_metric = dc.metrics.precision_score
    RC_metric = dc.metrics.recall_score
    F1_metric = dc.metrics.f1_score
    SP_metric = dc.metrics.recall_score


    for i in range(n_tasks):
        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true_train[:,i], y_tr_label[:,i][:,1],pos_label=1)
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i], average=None)[1]
        SP_score = SP_metric(y_true_train[:,i], y_tr_label[:,i][:,1],pos_label=0)
        
        result_df.loc[i,'Task']=tasks[i]
        result_df.loc[i,'Train AUC']=AUC_score
        result_df.loc[i,'Train ACC']=ACC_score
        result_df.loc[i,'Train F1']=F1_score
        result_df.loc[i,'Train Precision']=PR_score
        result_df.loc[i,'Train Recall']=RC_score    
        result_df.loc[i,'Train Specificity']=SP_score    

        
        
        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true_valid[:,i]),y_valid_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true_valid[:,i], y_valid_label[:,i][:,1],pos_label=1)
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid_label[:,i], average=None)[1]
        SP_score = SP_metric(y_true_valid[:,i], y_valid_label[:,i][:,1],pos_label=0)

        
        result_df.loc[i,'Valid AUC']=AUC_score
        result_df.loc[i,'Valid ACC']=ACC_score
        result_df.loc[i,'Valid F1']=F1_score
        result_df.loc[i,'Valid Precision']=PR_score
        result_df.loc[i,'Valid Recall']=RC_score    
        result_df.loc[i,'Valid Specificity']=SP_score    


        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i], average=None)[1]
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true[:,i], y_pred_label[:,i][:,1],pos_label=1)
        SP_score = SP_metric(y_true[:,i], y_pred_label[:,i][:,1],pos_label=0)

        
        result_df.loc[i,'Test AUC']=AUC_score
        result_df.loc[i,'Test ACC']=ACC_score
        result_df.loc[i,'Test F1']=F1_score
        result_df.loc[i,'Test Precision']=PR_score
        result_df.loc[i,'Test Recall']=RC_score
        result_df.loc[i,'Test Specificity']=SP_score    

    mean=result_df.mean()
    mean=mean.tolist()
    mean.insert(0,'AVG')
    column_headers = list(result_df.columns.values)
    for i in range(len(column_headers)):
        result_df.iloc[48,i]=mean[i]    
    result_df=result_df.dropna()
    end = time.time()
    print('Time: ',end - start)
    
    result_df.to_csv('DChem_FinalFishData_RB_MTL_LS421_'+str(e_n)+'.csv')  

In [ ]:
import tempfile
import deepchem as dc
import pandas as pd
from sklearn.metrics import precision_score
import tensorflow as tf
from tensorflow import keras
import time
import psutil
import warnings
warnings.filterwarnings("ignore")
import torch
import random
import tensorflow as tf
import numpy as np

set_seed(seed=8, tensorflow=True)
import tensorflow as tf

# List all available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    for gpu in gpus:
        # Allow GPU memory growth
        tf.config.experimental.set_memory_growth(gpu, True)

# Now, continue with your TensorFlow code



epoch_number=['500','10','100','200']

for e_n in epoch_number:
    print(e_n)
    

    start = time.time()

    
    ###result####
    number = list(range(0,500))    
    result_df=pd.DataFrame(index=number )
    col_list=['Task',
            'Train AUC', 'Train ACC','Train Balance ACC','Train F1','Train Precision', 'Train Recall','Train Specificity',
            'Valid AUC', 'Valid ACC','Valid Balance ACC','Valid F1','Valid Precision', 'Valid Recall','Valid Specificity',
            'Test AUC', 'Test ACC', 'Test Balance ACC','Test F1', 'Test Precision','Test Recall','Test Specificity']

    df=pd.read_csv('Zebrafish_final202310_withMorphologyALLdata.csv')
    tasks=['LEL_MORT','LEL_YSE','LEL_AXIS','LEL_EYE','LEL_SNOU', 'LEL_JAW',
        'LEL_OTIC','LEL_PE','LEL_BRAI','LEL_SOMI','LEL_PFIN','LEL_CFIN',
        'LEL_PIG','LEL_CIRC', 'LEL_TRUN','LEL_SWIM','LEL_NC','LEL_TR', '18_END_LEC', 'MOR_LEC','SUBLETH_17_END_LEC', 'TOX_SCO',
        'MO24', 'DP24', 'SM24', 'NC24','MORT','YSE', 'AXIS', 'EYE',
        'SNOU', 'JAW', 'OTIC', 'PE', 'BRAI', 'SOMI', 'PFIN', 'CFIN',
        'PIG','CIRC', 'TRUN', 'SWIM','NC','TR','MOV21', 'AUC21','METAB','MIC']
    #task=task.drop(columns='Unnamed: 0')
    labels=tasks
    for i in df.index:
        for q in labels:
            try:
                if df.loc[i,q]== 'ACT':
                    df.loc[i,q]=int(1)
                if df.loc[i,q]== 'INACT':           
                    df.loc[i,q]=int(0)             
            except:
                pass   

    print('ECFP6')
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:
      df.to_csv(tmpfile.name)
      loader = dc.data.CSVLoader(tasks, feature_field="SMILES",
                    featurizer=dc.feat.CircularFingerprint(size=1024, radius=3))
      dataset = loader.create_dataset(tmpfile.name)

    len(dataset)

    splitter = dc.splits.RandomStratifiedSplitter()
    # Splitting dataset into train and test datasets
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset,frac_train=0.7, frac_valid=0.1, frac_test=0.2,seed=8)

    #train_dataset, test_dataset = splitter.train_test_split(dataset,frac_train = 0.8,seed=8)


    n_tasks = len(tasks)
    n_features = train_dataset.get_data_shape()[0]
    
    print('train')
     # Create a MirroredStrategy.
    strategy = tf.distribute.MirroredStrategy()
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

    model = dc.models.ProgressiveMultitaskClassifier(n_tasks, n_features,
       mode="classification",       
        n_weave=2, 
       momentum= 0.9,
       learning_rate=0.0001,
       dropouts=[0.25,0.15,0.1],
       layer_sizes=[200,100,50],
            #[400,100,50]'
       penalty=0.005,
       weight_decay_penalty_type='l2',
       batch_normalize=False)

    e_n=int(e_n)
    # Open a strategy scope.
    for d in ['/device:GPU:0','/device:GPU:1','/device:GPU:2']:
        model.fit(train_dataset, nb_epoch=e_n)
    #######

    print('eval')
    y_true_train=train_dataset.y
    y_tr = model.predict(train_dataset)
    y_tr_label = (y_tr >= 0.5) 

    y_true_valid = valid_dataset.y
    y_valid = model.predict(valid_dataset)
    y_valid_label = (y_valid >= 0.5) 

    y_true = test_dataset.y
    y_pred = model.predict(test_dataset)
    y_pred_label = (y_pred >= 0.5) 

    AUC_metric = dc.metrics.roc_auc_score
    ACC_metric = dc.metrics.accuracy_score
    PR_metric = dc.metrics.precision_score
    RC_metric = dc.metrics.recall_score
    F1_metric = dc.metrics.f1_score
    SP_metric = dc.metrics.recall_score


    for i in range(n_tasks):
        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true_train[:,i], y_tr_label[:,i][:,1],pos_label=1)
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true_train[:,i]), y_tr_label[:,i], average=None)[1]
        SP_score = SP_metric(y_true_train[:,i], y_tr_label[:,i][:,1],pos_label=0)
        
        result_df.loc[i,'Task']=tasks[i]
        result_df.loc[i,'Train AUC']=AUC_score
        result_df.loc[i,'Train ACC']=ACC_score
        result_df.loc[i,'Train F1']=F1_score
        result_df.loc[i,'Train Precision']=PR_score
        result_df.loc[i,'Train Recall']=RC_score    
        result_df.loc[i,'Train Specificity']=SP_score    

        
        
        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true_valid[:,i]),y_valid_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true_valid[:,i], y_valid_label[:,i][:,1],pos_label=1)
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true_valid[:,i]), y_valid_label[:,i], average=None)[1]
        SP_score = SP_metric(y_true_valid[:,i], y_valid_label[:,i][:,1],pos_label=0)

        
        result_df.loc[i,'Valid AUC']=AUC_score
        result_df.loc[i,'Valid ACC']=ACC_score
        result_df.loc[i,'Valid F1']=F1_score
        result_df.loc[i,'Valid Precision']=PR_score
        result_df.loc[i,'Valid Recall']=RC_score    
        result_df.loc[i,'Valid Specificity']=SP_score    


        AUC_score = AUC_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
        ACC_score = ACC_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i])
        PR_score = PR_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i], average=None)[1]
        F1_score = F1_metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_label[:,i], average=None)[1]
        RC_score = RC_metric(y_true[:,i], y_pred_label[:,i][:,1],pos_label=1)
        SP_score = SP_metric(y_true[:,i], y_pred_label[:,i][:,1],pos_label=0)

        
        result_df.loc[i,'Test AUC']=AUC_score
        result_df.loc[i,'Test ACC']=ACC_score
        result_df.loc[i,'Test F1']=F1_score
        result_df.loc[i,'Test Precision']=PR_score
        result_df.loc[i,'Test Recall']=RC_score
        result_df.loc[i,'Test Specificity']=SP_score    

    mean=result_df.mean()
    mean=mean.tolist()
    mean.insert(0,'AVG')
    column_headers = list(result_df.columns.values)
    for i in range(len(column_headers)):
        result_df.iloc[48,i]=mean[i]    
    result_df=result_df.dropna()
    end = time.time()
    print('TIME: ',end - start)

    
    result_df.to_csv('DChem_FinalFishData_PRMTL_LS215_'+str(e_n)+'.csv')  